**Multi-qubit gate reference**
![circuit](./images/I.14.1.1.png)
![circuit](./images/I.14.1.2.png)
![circuit](./images/I.14.1.3.png)
![circuit](./images/I.14.1.4.png)

**Single-qubit gate reference**
![circuit](./images/I.14.1.5.png)
![circuit](./images/I.14.1.6.png)
![circuit](./images/I.14.1.7.png)

In [1]:
import numpy as np
import pennylane as qml

**Codercise I.14.1** Consider again the entangled state that we saw earlier,
![circuit](./images/I.14.1.8.png)
This state is called a **Bell state**, and it has 3 siblings:
![circuit](./images/I.14.1.9.png)
Together, these states form the **Bell basis**. Write a set of 4 circuits that prepare and return each of the four Bell states.

In [2]:
dev = qml.device('default.qubit', wires=2)

# Starting from the state |00>, implement a PennyLane circuit
# to construct each of the Bell basis states.

@qml.qnode(dev)
def prepare_psi_plus():
    # PREPARE (1/sqrt(2)) (|00> + |11>)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])

    return qml.state()


@qml.qnode(dev)
def prepare_psi_minus():
    # PREPARE (1/sqrt(2)) (|00> - |11>)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.PauliZ(wires=0)

    # alternative approach
    # qml.PauliX(wires=0)
    # qml.Hadamard(wires=0)
    # qml.CNOT(wires=[0,1])

    return qml.state()


@qml.qnode(dev)
def prepare_phi_plus():
    # PREPARE  (1/sqrt(2)) (|01> + |10>)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.PauliX(wires=1)

    # alternative approach
    # qml.PauliX(wires=1)
    # qml.Hadamard(wires=0)
    # qml.CNOT(wires=[0,1])

    return qml.state()


@qml.qnode(dev)
def prepare_phi_minus():
    # PREPARE  (1/sqrt(2)) (|01> - |10>)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.PauliZ(wires=0)
    qml.PauliX(wires=1)

    # alternative approach
    # qml.PauliX(wires=0)
    # qml.PauliX(wires=1)
    # qml.Hadamard(wires=0)
    # qml.CNOT(wires=[0,1])

    return qml.state()


psi_plus = prepare_psi_plus()
psi_minus = prepare_psi_minus()
phi_plus = prepare_phi_plus()
phi_minus = prepare_phi_minus()

# Uncomment to print results
#print(f"|ψ_+> = {psi_plus}")
#print(f"|ψ_-> = {psi_minus}")
#print(f"|ϕ_+> = {phi_plus}")
#print(f"|ϕ_-> = {phi_minus}")

![circuit](./images/I.14.1.10.png)

Codercise I.14.2. Implement a 3-qubit circuit in PannyLane that can perform the following:
* if the first two qubits are both $|0>$, do nothing
* if the first qubit is $|0>$ and the second is $|1>$, apply `PauliX` to the third qubit
* if the first qubit is $|1>$ and the second is $|0>$, apply `PauliZ` to the third qubit
* if the two qubit are both $|1>$, apply a `PauliY` the third qubit

The circuit must produce the exact state that would be obtained by applying these operations, i.e., not just up to a global phase.
There is no nedd to use `if` statements in your part of the quantum function; it can all be implemented using quantum operations alone!
Tip. This type of operation is called **quantum multiplexer**. When all $2^n$ possible cases of $n$ control qubits are implemented, and the target operation is a single-qubit roation, it is called a uniformly controlled rotation.

In [4]:
dev = qml.device("default.qubit", wires=3)

# State of first 2 qubits
state = [0, 1]

@qml.qnode(device=dev)
def apply_control_sequence(state):
    # Set up initial state of the first two qubits
    if state[0] == 1:
        qml.PauliX(wires=0)
    if state[1] == 1:
        qml.PauliX(wires=1)

    # Set up initial state of the third qubit - use |->
    # so we can see the effect on the output
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)


    # IMPLEMENT THE MULTIPLEXER
    # IF STATE OF FIRST TWO QUBITS IS 01, APPLY X TO THIRD QUBIT
    qml.MultiControlledX(wires=[0,1,2], control_values="01")

    # IF STATE OF FIRST TWO QUBITS IS 10, APPLY Z TO THIRD QUBIT
    qml.Hadamard(wires=2)
    qml.MultiControlledX(wires=[0,1,2], control_values="10")
    qml.Hadamard(wires=2)

    # IF STATE OF FIRST TWO QUBITS IS 11, APPLY Y TO THIRD QUBIT
    qml.adjoint(qml.S)(wires=2)
    qml.Toffoli(wires=[0,1,2])
    qml.S(wires=2)

    # Proposed solution by codebook
    # qml.PauliX(wires=0)
    # qml.Toffoli(wires=[0,1,2])
    # qml.PauliX(wires=0)

    # qml.PauliX(wires=1)
    # qml.Hadamard(wires=2)
    # qml.Toffoli(wires=[0,1,2])
    # qml.PauliX(wires=1)
    # qml.Hadamard(wires=2)

    # qml.adjoint(qml.S)(wires=2)
    # qml.Toffoli(wires=[0,1,2])
    # qml.S(wires=2)

    return qml.state()


print(apply_control_sequence(state))

[ 0.        +0.j  0.        +0.j -0.70710678+0.j  0.70710678+0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]


<em>Proposed Solution by codebook</em>
All of these operations can be expressed as controlled operations, but controlled on different states. Since each operation will 'trigger' only on a specific input state, we can apply them one after another in a circuit:
![circuit](./images/I.14.2.1.png)

We now need to re-express, or compile these gates in terms of the Toffoli available in PennyLane.

For the first gate, a controlled-controlled-X is simply a Toffoli. Since one of the controls is controlling on 0, we must apply an **X** both before and after the controls.

For the second gate, recall that $Z=HXH$, and that a controlled $Z$ consisted of $H$. Thus, a Toffoli has a similar identity, and a controlled-controlled-Z (CCZ) can be implemented using a Toffoli and aplication of $H$ on both sides of the target qubit. Again, since one of the controls i $0$, we have to apply $X$ before and after.

For the final gate, we need the identity $Y = SXS^*$, meaning we can implement a  controlled-controlled-Y by applying $S^*$ and $S$ before and after a toffoli respectively. Our final circuit is thus:
![circuit](./images/I.14.2.2.png)